# MariaDB with Python Exercises

Make sure that you finished the Basic SQL Exercises before starting this exercise because you will be converting what you have already done from manual inputs to using Python and MariaDB's Python connector to interface directly with MariaDBMS. You are still encouraged to use MySQL Client to help check your work.

---

**Q1)** Write SQL statements to answer the questions using the following table.

Table name: salesman

| salesman_id |    name    |   city   | commission |
|------------:|:-----------|:---------|-----------:|
|        5001 | James Hoog | New York |       0.15 |
|        5002 | Nail Knite | Paris    |       0.13 |
|        5005 | Pit Alex   | London   |       0.11 |
|        5006 | Mc Lyon    | Paris    |       0.14 |
|        5007 | Paul Adam  | Rome     |       0.13 |
|        5003 | Lauson Hen | San Jose |       0.12 |
        
1. Return all the salesman data for salesman that either comes from the city Paris or London.
2. Find the average commission for salesmen from Paris.
3. Find the salesman ids of those salesmen with names starting with the letter "P".

In [2]:
# imports mariadb library
import mariadb

try:
    # Instantiate Connection
    conn = mariadb.connect(
        user="root",
        password="",
        host="localhost",
        port=3306)
    print("Success")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")

Success


In [3]:
cur = conn.cursor() 

In [4]:
# creating database  
cur.execute("CREATE DATABASE sales")

ProgrammingError: Can't create database 'sales'; database exists

In [6]:
import mariadb_context as rdbms

In [9]:
with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
    rdbms_cursor.execute("SELECT * FROM salesman WHERE city='London' OR city ='Paris';")
    
    rows = rdbms_cursor.fetchall()
    
    for row in rows:
        print(row)

('5002', 'Nail Knite', 'Paris', Decimal('0.13'))
('5005', 'Pit Alex', 'London', Decimal('0.11'))
('5006', 'Mc Lyon', 'Paris', Decimal('0.14'))


In [10]:
with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
    rdbms_cursor.execute("SELECT AVG(commission) FROM salesman WHERE city ='Paris';")
    
    rows = rdbms_cursor.fetchall()
    
    for row in rows:
        print(row)

(Decimal('0.135000'),)


In [24]:
with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
    rdbms_cursor.execute('SELECT salesman_id, name FROM salesman WHERE name LIKE "P%";')    
    rows = rdbms_cursor.fetchall()
    
    for row in rows:
        print(row)

('5005', 'Pit Alex')
('5007', 'Paul Adam')


---
**Q2)** Write SQL statements to answer the questions using the following table. 

Table name: salesman

| salesman_id |    name    |   city   | commission | 
|------------:|:-----------|:---------|-----------:|
|        5001 | James Hoog | New York |       0.15 |
|        5002 | Nail Knite | Paris    |       0.13 |
|        5005 | Pit Alex   | London   |       0.11 |
|        5006 | Mc Lyon    | Paris    |       0.14 |
|        5007 | Paul Adam  | Rome     |       0.13 |
|        5003 | Lauson Hen | San Jose |       0.12 |
        
Table name: customer

| cust_id     |   cust_name    |    city    | grade | salesman_id |
|------------:|:---------------|:-----------|------:|------------:|
|        3002 | Nick Rimando   | New York   |   100 |        5001 |
|        3007 | Brad Davis     | New York   |   200 |        5001 |
|        3005 | Graham Zusi    | California |   200 |        5002 |
|        3008 | Julian Green   | London     |   300 |        5002 |
|        3004 | Fabian Johnson | Paris      |   300 |        5006 |
|        3009 | Geoff Cameron  | Berlin     |   100 |        5003 |
|        3003 | Jozy Altidor   | Moscow     |   200 |        5007 |
|        3001 | Brad Guzan     | London     |   250 |        5005 |
        
1. Return both the customer's and salesmen's name and id who lives in the same city.
2. Return the salesman's name and id where customers have graded them over 250.
3. Customer Brad Davis have moved from New York to Hong Kong, update the customer's data to reflect this change.
4. Using the following data, insert the following customer records:
 <pre>
 {'new_customers':
     ({'cust_id': 3006, 'cust_name': 'Joe Black', 'city': 'California', 'grade': 200, 'attendant_salesman':'Pit Alex'},
      {'cust_id': 3010, 'cust_name': 'Blue Lightning', 'city': 'Hamburg', 'grade': 300, 'attendant_salesman':'Paul Adam'},
      {'cust_id': 3011, 'cust_name': 'Lyra Slivertongue', 'city': 'Oxford', 'grade': 300, 'attendant_salesman':'Nail Knite'}
     )}
 </pre>
 you are to use the dictionary data as it is.

In [25]:
with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
    rdbms_cursor.execute('SELECT cust_name, name FROM customer, salesman WHERE customer.city = salesman.city;')    
    rows = rdbms_cursor.fetchall()
    
    for row in rows:
        print(row)

('Brad Guzan', 'Pit Alex')
('Nick Rimando', 'James Hoog')
('Fabian Johnson', 'Nail Knite')
('Fabian Johnson', 'Mc Lyon')
('Julian Green', 'Pit Alex')


In [26]:
with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
    rdbms_cursor.execute('SELECT salesman.name, salesman.salesman_id FROM salesman, customer WHERE customer.grade > 250 AND salesman.salesman_id = customer.salesman_id;')    
    rows = rdbms_cursor.fetchall()
    
    for row in rows:
        print(row)

('Mc Lyon', '5006')
('Nail Knite', '5002')


In [28]:
with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
    rdbms_cursor.execute('UPDATE customer AS c SET city="Hong Kong" WHERE c.cust_name = "Brad Davis";')    
    
    rdbms_cursor.execute('SELECT * FROM customer')
    
    rows = rdbms_cursor.fetchall()
    
    for row in rows:
        print(row)

('3001', 'Brad Guzan', 'London', 200, '5005')
('3002', 'Nick Rimando', 'New York', 100, '5001')
('3003', 'Jozy Altidor', 'Moscow', 200, '5007')
('3004', 'Fabian Johnson', 'Paris', 300, '5006')
('3005', 'Graham Zusi', 'California', 200, '5002')
('3007', 'Brad Davis', 'Hong Kong', 200, '5001')
('3008', 'Julian Green', 'London', 300, '5002')
('3009', 'Geoff Cameron', 'Berlin', 100, '5003')


In [60]:
d = {'new_customers':
  ({'cust_id': 3006, 'cust_name': 'Joe Black', 'city': 'California', 'grade': 200, 'attendant_salesman':'Pit Alex'},
   {'cust_id': 3010, 'cust_name': 'Blue Lightning', 'city': 'Hamburg', 'grade': 300, 'attendant_salesman':'Paul Adam'},
   {'cust_id': 3011, 'cust_name': 'Lyra Slivertongue', 'city': 'Oxford', 'grade': 300, 'attendant_salesman':'Nail Knite'}
  )}

# with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
#     for di in d['new_customers']:
#         rdbms_cursor.execute(f'(SELECT salesman_id FROM salesman WHERE name = {di["attendant_name"]});')

with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
    for di in d['new_customers']:
#         rdbms_cursor.execute(f"""SELECT salesman_id FROM salesman WHERE name = "{di["attendant_salesman"]}";""")
#         salesman_id = rdbms_cursor.fetchone()
#         print(salesman_id)
#         rdbms_cursor.execute(f"""INSERT INTO customer VALUES (?,?,?,?,(SELECT salesman_id FROM salesman WHERE name=?));""", (di["cust_id"],
#                                                                              di["cust_name"],
#                                                                              di["city"],
#                                                                              di["grade"],
#                                                                              di['attendant_salesman']))
    
        rdbms_cursor.execute(f"""INSERT INTO customer VALUES ({di["cust_id"]},
            '{di["cust_name"]}',
            '{di["city"]}',
            {di["grade"]},
            (SELECT salesman_id FROM salesman WHERE name='{di['attendant_salesman']}'));""")

# LESSON LEARNT: Must use ?,?,?,? and use the params tuple parameter to replace the ? with the parameter respectively.
# LESSON LEARNT: For the parameters that are returning string, MUST ENCLOSE IT WITH SINGLE QUOTES.
# LESSON LEARNT: YOU CAN COMPOUND SQL STATEMENT IN ANOTHER SQL STATEMENT USING PARENTHESIS!!!

IntegrityError: Duplicate entry '3006' for key 'PRIMARY'

In [51]:
# Chee Wai's Answers

dict = {'new_customers':
  ({'cust_id': 3006, 'cust_name': 'Joe Black', 'city': 'California', 'grade': 200, 'attendant_salesman':'Pit Alex'},
   {'cust_id': 3010, 'cust_name': 'Blue Lightning', 'city': 'Hamburg', 'grade': 300, 'attendant_salesman':'Paul Adam'},
   {'cust_id': 3011, 'cust_name': 'Lyra Slivertongue', 'city': 'Oxford', 'grade': 300, 'attendant_salesman':'Nail Knite'}
  )}

# with rdbms.MariaDBCursor("", "sales") as rdbms_cursor:
#     for di in d['new_customers']:
#         rdbms_cursor.execute(f'(SELECT salesman_id FROM salesman WHERE name = {di["attendant_name"]});')

with rdbms.MariaDBCursor("", "sales") as rc:
    for d in dict['new_customers']:
            rc.execute("insert into customer values (?,?,?,?, (select salesman_id from salesman where name=?))", 
                       (d['cust_id'],
                        d['cust_name'],
                        d['city'],
                        d['grade'],
                        d['attendant_salesman']
                       )
                      ) 
    

IntegrityError: Duplicate entry '3006' for key 'PRIMARY'

---
**Q3)** Given the following database schema (from lecture notes) below, answer the following questions

**Student**(sID CHAR(8), sName CHAR(50), gender CHAR(1), age INT, dID CHAR(2), grade CHAR(2))    
**Dept**(dID CHAR(2), dName CHAR(20), dean CHAR(50))   
**Course**(cID CHAR(3), cName CHAR(50), hours INT, credit INT, iID CHAR(3))   
**Instructor**(iID CHAR(3), iName CHAR(50), dID CHAR(2), workload FLOAT)   
**RC**(sID CHAR(8), cID CHAR(3), score FLOAT)    

1. Find the names of the students who are taking the courses taught by "Lim".
2. Using the function aggregate function `MAX()`, find the oldest female student in the CS department.
3. Using the not equal to operator (`<>`) and the `ORDER BY` clause, find all the students who are not from the CS department and group them by department.
4. Using the `GROUP BY` clause, return the avg age of the students for each department.